In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# import numpy as np
# import pandas as pd

In [ ]:
# pd.read_csv('/content/drive/My Drive/1 Data/yoochoose_click_binary_100000_sess.dataset')

UnicodeDecodeError: ignored

In [ ]:
!pip install torch-geometric \
  torch-sparse==latest+cu101 \
  torch-scatter==latest+cu101 \
  torch-cluster==latest+cu101 \
  -f https://pytorch-geometric.com/whl/torch-1.5.0.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 153kB 2.8MB/s 
     |████████████████████████████████| 21.6MB 1.2MB/s 
     |████████████████████████████████| 12.3MB 249kB/s 
     |████████████████████████████████| 18.2MB 168kB/s 
     |████████████████████████████████| 235kB 7.9MB/s 
     |████████████████████████████████| 2.1MB 11.4MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.5.0-cp36-none-any.whl size=267918 sha256=7f17d8102cb513836afda599154f4bf07c37f6e4384c2bc3ba819b8574e87609
  Stored in directory: /root/.cache/pip/wheels/ec/51/31/5786f2ac419ee312f22d4d2877da05f20e7f2d430e22917daf
Successfully built torch-geometric


In [ ]:
import numpy as np
import pandas as pd
import pickle
import csv
import os
import torch
from torch_geometric.data import Data
from tqdm import tqdm
np.random.seed(42)

In [ ]:
import os
os.chdir('/content/drive/My Drive/RecSys')

FileNotFoundError: ignored

In [ ]:
os.listdir('/content/drive/My Drive/1_RecSys')

['yoochoose-buys.dat',
 'yoochoose-clicks.dat',
 'yoochoose-test.dat',
 '.ipynb_checkpoints',
 'yoochoose_click_binary_100000_sess.dataset']

In [ ]:
# df = pd.read_csv('/content/drive/My Drive/RecSys/yoochoose_click_binary_100000_sess.dataset', header=None)
# df
# df.columns=['session_id','timestamp','item_id','category']

In [ ]:
df = pd.read_csv('/content/drive/My Drive/1_RecSys/yoochoose-clicks.dat', header=None)
# df
df.columns=['session_id','timestamp','item_id','category']

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.head()

,session_id,timestamp,item_id,category
0,1,2014-04-07T10:51:09.277Z,214536502,0
1,1,2014-04-07T10:54:09.868Z,214536500,0
2,1,2014-04-07T10:54:46.998Z,214536506,0
3,1,2014-04-07T10:57:00.306Z,214577561,0
4,2,2014-04-07T13:56:37.614Z,214662742,0


In [ ]:
df['session_id'].min(),df['session_id'].max()

(1, 11562161)

In [ ]:
df.shape

(33003944, 4)

In [ ]:
buy_df = pd.read_csv('/content/drive/My Drive/1_RecSys/yoochoose-buys.dat', header=None)
buy_df.columns=['session_id','timestamp','item_id','price','quantity']
buy_df.head()

,session_id,timestamp,item_id,price,quantity
0,420374,2014-04-06T18:44:58.314Z,214537888,12462,1
1,420374,2014-04-06T18:44:58.325Z,214537850,10471,1
2,281626,2014-04-06T09:40:13.032Z,214535653,1883,1
3,420368,2014-04-04T06:13:28.848Z,214530572,6073,1
4,420368,2014-04-04T06:13:28.858Z,214835025,2617,1


In [ ]:
buy_df.shape

(1150753, 5)

In [ ]:
buy_df['session_id'].min(),buy_df['session_id'].max()

(11, 11562121)

In [ ]:
df [df['session_id'] == 281626]

,session_id,timestamp,item_id,category
824193,281626,2014-04-06T09:20:42.331Z,214535653,0
824194,281626,2014-04-06T09:22:05.402Z,214819357,0
824195,281626,2014-04-06T09:22:58.720Z,214535653,0
824196,281626,2014-04-06T09:23:15.534Z,214535653,0
824197,281626,2014-04-06T09:24:16.421Z,214821277,0
824198,281626,2014-04-06T09:26:07.764Z,214684513,0
824199,281626,2014-04-06T09:28:15.255Z,214535681,0
824200,281626,2014-04-06T09:29:00.938Z,214552370,0
824201,281626,2014-04-06T09:31:52.830Z,214698577,0


In [ ]:
df.groupby('session_id')['item_id'].size()

session_id
1           4
2           6
3           3
4           2
6           2
           ..
11562156    2
11562157    2
11562158    3
11562159    1
11562161    1
Name: item_id, Length: 9249729, dtype: int64

In [ ]:
df['valid_session'] = df.session_id.map(df.groupby('session_id')['item_id'].size() > 2)
df['valid_session'].value_counts()

True     24628059
False     8375885
Name: valid_session, dtype: int64

In [ ]:
df.loc[df.valid_session].shape

(24628059, 5)

In [ ]:
# filter out item session with length > 2
df['valid_session'] = df.session_id.map(df.groupby('session_id')['item_id'].size() > 2)
df = df.loc[df.valid_session].drop('valid_session',axis=1)
df.shape

(24628059, 4)

In [ ]:
df [df['session_id'] == 4726778]

,session_id,timestamp,item_id,category


In [ ]:
df.nunique()

session_id     4431931
timestamp     24590089
item_id          48255
category           331
dtype: int64

In [ ]:
# #randomly sample a couple of them
sampled_session_id = np.random.choice(df.session_id.unique(), 100000, replace=False)
df = df.loc[df.session_id.isin(sampled_session_id)]
df.nunique()

session_id    100000
timestamp     555860
item_id        22875
category         136
dtype: int64

In [ ]:
# average length of session 
df.groupby('session_id')['item_id'].size().mean()

5.5589

In [ ]:
from sklearn.preprocessing import LabelEncoder

item_encoder = LabelEncoder()
category_encoder = LabelEncoder()
df['item_id'] = item_encoder.fit_transform(df.item_id )
df['category']= category_encoder.fit_transform(df.category.apply(str))
df.head()

,session_id,timestamp,item_id,category
341,108,2014-04-03T11:29:34.491Z,5446,0
342,108,2014-04-03T11:30:31.064Z,5447,0
343,108,2014-04-03T11:31:36.145Z,5447,0
616,194,2014-04-04T14:29:16.071Z,15871,0
617,194,2014-04-04T14:29:47.845Z,15871,0


In [ ]:
df['item_id'].min(),df['item_id'].max()

(0, 22874)

In [ ]:
#  check there is any buy event for a given session
buy_df = buy_df.loc[buy_df.session_id.isin(df.session_id)]
buy_df['item_id'] = item_encoder.transform(buy_df.item_id)
buy_df.head()

,session_id,timestamp,item_id,price,quantity
101,420508,2014-04-07T15:37:26.815Z,10020,1046,1
102,420508,2014-04-07T15:37:26.896Z,7907,1570,1
103,420508,2014-04-07T15:37:26.950Z,15606,627,1
106,281747,2014-04-06T10:32:23.948Z,16075,523,2
107,281747,2014-04-06T10:32:23.984Z,16075,523,1


In [ ]:
buy_df['session_id'].min(),buy_df['session_id'].max()

(664, 11561264)

In [ ]:
buy_df['item_id'].min(),buy_df['item_id'].max()

(0, 22874)

In [ ]:
buy_item_dict = dict(buy_df.groupby('session_id')['item_id'].apply(list))
buy_item_dict

{664: [7370],
 3074: [16928, 13598, 11040, 11040],
 3852: [10119],
 4436: [1057],
 4759: [15490, 15717],
 4982: [9725, 10495],
 6223: [4810],
 6839: [4665],
 10836: [15658, 2136],
 12593: [5822],
 12874: [13835, 13826],
 13159: [4806],
 13332: [15530],
 13726: [9367],
 14314: [13862, 15596, 15526, 15714],
 14819: [15719, 15493],
 18166: [7034],
 18577: [7486],
 18744: [50],
 21441: [269],
 22427: [16103, 16088],
 23912: [13831],
 25232: [10435],
 26399: [16117, 16155, 16112, 16145, 16277, 16143, 17974],
 27157: [12373, 17125],
 27814: [8985],
 28444: [4814],
 29269: [15666],
 30136: [12175, 12925, 12904, 14183],
 30416: [12672],
 31394: [13831],
 34594: [15824, 3479, 12876, 12874, 12876, 14185],
 34787: [13862, 13827],
 35436: [15720, 15438],
 35903: [15588, 14698],
 38431: [15823, 15872],
 39811: [15823, 15827],
 41699: [13837],
 42681: [18395, 16075],
 44522: [5446],
 45246: [15669, 15659],
 45301: [16091, 16110],
 46324: [5573],
 46489: [1397],
 47369: [9352],
 51357: [16079],
 5154

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
if 467077 in buy_item_dict:
    print("Hello")
    positive_indices = le.fit_transform(buy_item_dict[467077])
    print(positive_indices)


In [ ]:
df.head()

,session_id,timestamp,item_id,category
341,108,2014-04-03T11:29:34.491Z,5446,0
342,108,2014-04-03T11:30:31.064Z,5447,0
343,108,2014-04-03T11:31:36.145Z,5447,0
616,194,2014-04-04T14:29:16.071Z,15871,0
617,194,2014-04-04T14:29:47.845Z,15871,0


In [ ]:
import torch
from torch_geometric.data import InMemoryDataset
from tqdm import tqdm

class YooChooseDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(YooChooseDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []
    @property
    def processed_file_names(self):
        return ['/content/drive/My Drive/1_RecSys/yoochoose_click_binary_100000_sess.dataset']

    def download(self):
        pass
    
    def process(self):
        
        data_list = []

        # process by session_id
        grouped = df.groupby('session_id')
        for session_id, group in tqdm(grouped):
            le = LabelEncoder()
            sess_item_id = le.fit_transform(group.item_id)
            group = group.reset_index(drop=True)
            group['sess_item_id'] = sess_item_id
            node_features = group.loc[group.session_id==session_id,['sess_item_id','item_id','category']].sort_values('sess_item_id')[['item_id','category']].drop_duplicates().values

            node_features = torch.LongTensor(node_features).unsqueeze(1)
            target_nodes = group.sess_item_id.values[1:]
            source_nodes = group.sess_item_id.values[:-1]

            edge_index = torch.tensor([source_nodes,
                                   target_nodes], dtype=torch.long)
            x = node_features

            if session_id in buy_item_dict:
                positive_indices = le.transform(buy_item_dict[session_id])
                # print(positive_indices)
                label = np.zeros(len(node_features))
                label[positive_indices] = 1
            else:
                # print("Hello")
                label = [0] * len(node_features)


            y = torch.FloatTensor(label)

            data = Data(x=x, edge_index=edge_index, y=y)

            data_list.append(data)
        
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])
        

In [ ]:
dataset = YooChooseDataset('../')

In [ ]:
dataset

YooChooseDataset(100000)

In [ ]:
# grouped = df.groupby('session_id')
# data_list= []
# for session_id, group in tqdm(grouped):
#     le = LabelEncoder()
#     sess_item_id = le.fit_transform(group.item_id)
#     group = group.reset_index(drop=True)
#     group['sess_item_id'] = sess_item_id
#     node_features = group.loc[group.session_id==session_id,['sess_item_id','item_id']].sort_values('sess_item_id').item_id.drop_duplicates().values

#     node_features = torch.LongTensor(node_features).unsqueeze(1)
#     target_nodes = group.sess_item_id.values[1:]
#     source_nodes = group.sess_item_id.values[:-1]

#     edge_index = torch.tensor([source_nodes,
#                            target_nodes], dtype=torch.long)
#     x = node_features
    
#     if session_id in buy_item_dict:
#         positive_indices = le.transform(buy_item_dict[session_id])
#         label = np.zeros(len(node_features))
#         label[positive_indices] = 1
#     else:
#         label = [0] * len(node_features)
#     y = torch.FloatTensor(label)
#     data = Data(x=x, edge_index=edge_index, y=y)
#     data_list.append(data)

#


In [ ]:
dataset = dataset.shuffle()
one_tenth_length = int(len(dataset) * 0.1)
train_dataset = dataset[:one_tenth_length * 8]
val_dataset = dataset[one_tenth_length*8:one_tenth_length * 9]
test_dataset = dataset[one_tenth_length*9:]
len(train_dataset), len(val_dataset), len(test_dataset)

(80000, 10000, 10000)

In [ ]:
from torch_geometric.data import DataLoader
batch_size= 512
train_loader = DataLoader(train_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
num_items = df.item_id.max() +1
num_categories = df.category.max()+1
num_items , num_categories

(22875, 135)

In [ ]:
embed_dim = 128
from torch_geometric.nn import GraphConv, TopKPooling, GatedGraphConv, SAGEConv, SGConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = GraphConv(embed_dim * 2, 128)
        self.pool1 = TopKPooling(128, ratio=0.9)
        self.conv2 = GraphConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.9)
        self.conv3 = GraphConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.9)
        self.item_embedding = torch.nn.Embedding(num_embeddings=num_items, embedding_dim=embed_dim)
        self.category_embedding = torch.nn.Embedding(num_embeddings=num_categories, embedding_dim=embed_dim)        
        self.lin1 = torch.nn.Linear(256, 256)
        self.lin2 = torch.nn.Linear(256, 128)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()        
  
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        item_id = x[:,:,0]
        category = x[:,:,1]
        

        emb_item = self.item_embedding(item_id).squeeze(1)
        emb_category = self.category_embedding(category).squeeze(1)
        
#         emb_item = emb_item.squeeze(1)
#         emb_cat
        x = torch.cat([emb_item, emb_category], dim=1)  
#         print(x.shape)
        x = F.relu(self.conv1(x, edge_index))
#                 print(x.shape)
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
     
        x, edge_index, _, batch, _, _= self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        x, edge_index, _, batch, _, _= self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.act2(x)      
        
        outputs = []
        for i in range(x.size(0)):
            output = torch.matmul(emb_item[data.batch == i], x[i,:])

            outputs.append(output)
              
        x = torch.cat(outputs, dim=0)
        x = torch.sigmoid(x)
        
        return x

In [ ]:
device = torch.device('cuda')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
crit = torch.nn.BCELoss()

In [ ]:
def train():
    model.train()

    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)

        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)

In [ ]:
from sklearn.metrics import roc_auc_score
def evaluate(loader):
    model.eval()

    predictions = []
    labels = []

    with torch.no_grad():
        for data in loader:

            data = data.to(device)
            pred = model(data).detach().cpu().numpy()

            label = data.y.detach().cpu().numpy()
            predictions.append(pred)
            labels.append(label)

    predictions = np.hstack(predictions)
    print(predictions)
    labels = np.hstack(labels)
    print(labels)
    
    return roc_auc_score(labels, predictions)

In [ ]:
# print(predictions)

In [ ]:
for epoch in range(1, 10):
    loss = train()
    # val_loader = DataLoader(val_dataset, batch_size=batch_size)
    # test_loader = DataLoader(test_dataset, batch_size=batch_size)   
    train_acc = evaluate(train_loader)
    val_acc = evaluate(val_loader)    
    test_acc = evaluate(test_loader)
    print('Epoch: {:03d}, Loss: {:.5f}, Train Auc: {:.5f}, Val Auc: {:.5f}, Test Auc: {:.5f}'.
          format(epoch, loss, train_acc, val_acc, test_acc))

[0.16560917 0.07790209 0.10135177 ... 0.01971406 0.06365018 0.00840537]
[0. 0. 0. ... 0. 0. 0.]
[0.01152095 0.2196879  0.10610659 ... 0.10082365 0.22960475 0.53786516]
[0. 0. 0. ... 0. 0. 0.]
[0.11317858 0.21758445 0.43429485 ... 0.5368034  0.22449881 0.6288844 ]
[0. 0. 0. ... 0. 0. 0.]
Epoch: 001, Loss: 0.58492, Train Auc: 0.55158, Val Auc: 0.53324, Test Auc: 0.53015
[0.05321203 0.02448004 0.03867444 ... 0.00037027 0.07113855 0.00156219]
[0. 0. 0. ... 0. 0. 0.]
[0.02891396 0.03593655 0.01710921 ... 0.03900605 0.09587044 0.6950277 ]
[0. 0. 0. ... 0. 0. 0.]
[0.01669218 0.11511355 0.27758098 ... 0.35670117 0.08262093 0.5606728 ]
[0. 0. 0. ... 0. 0. 0.]
Epoch: 002, Loss: 0.39018, Train Auc: 0.59858, Val Auc: 0.56173, Test Auc: 0.56133
[0.04527156 0.02506711 0.02063427 ... 0.00037896 0.01275631 0.00030528]
[0. 0. 0. ... 0. 0. 0.]
[0.0260963  0.01100675 0.02184006 ... 0.01873104 0.09212289 0.6193007 ]
[0. 0. 0. ... 0. 0. 0.]
[0.00225774 0.12654947 0.09139088 ... 0.27062407 0.05024446 0.4960